# Hopf Bifurcation Study

In [ ]:
from tqdm.notebook import tqdm
from os.path import isfile, splitext

import h5py
import numpy as np
import jax
from jax import numpy as jnp
import matplotlib.pyplot as plt

from femvf import load, statefile as sf, meshutils
from femvf.models.transient import solid as smd, fluid as fmd
from femvf.signals.solid import make_sig_glottal_width_sharp
from blocktensor import h5utils as bh5utils
from vfsig import modal
from vfsig.misc import resample_over_uniform_period

import libhopf
import main_hopf
import libsignal
from lib_main_transient import case_config
import h5utils
from postprocutils import postprocess_case_to_signal

In [ ]:
## Global vars
ZETA = 1e-4
R_SEP = 1.0
Y_GAP = 1e-2
PSUBS = np.concatenate([np.arange(200, 300, 10), np.arange(300, 1000, 100)])*10
ECOV = 5e3 * 10
EBODY = 15e3 * 10
EBODY = 5e3 * 10

# load transient model
INIT_STATE_TYPE = 'static'

# OUT_DIR = f'out/zeta{ZETA:.2e}_rsep{R_SEP:.1f}_ygap{Y_GAP:.2e}_init{INIT_STATE_TYPE}_fixed_rsep'
# FluidType = fmd.BernoulliMinimumSeparation
OUT_DIR = f'out/zeta{ZETA:.2e}_rsep{R_SEP:.1f}_ygap{Y_GAP:.2e}_init{INIT_STATE_TYPE}_variable_rsep'
FluidType = fmd.Bernoulli


mesh_name = 'BC-dcov5.00e-02-cl1.00'
mesh_path = f'mesh/{mesh_name}.xml'
model = load.load_transient_fsi_model(
    mesh_path, None, 
    SolidType=smd.KelvinVoigt, 
    FluidType=FluidType, 
    coupling='explicit'
)

# load Hopf model
res, dres = main_hopf.setup_models(mesh_path)
region_to_dofs = meshutils.process_celllabel_to_dofs_from_forms(
    res.solid.forms, res.solid.forms['fspace.scalar'])
props = main_hopf.set_properties(res.properties.copy(), region_to_dofs, res)
res.set_properties(props)
dres.set_properties(props)

## Post-processing

In [ ]:
## Initialize functions to extract glottal width from saved simulations
N_PER_PERIOD = 100
glottal_width_tran = make_sig_glottal_width_sharp(model)
glottal_width_hopf = libsignal.make_glottal_width(res, dres, num_points=N_PER_PERIOD)
def time(f):
    return np.array(f.get_times())

In [ ]:
## Post processing transient simulations
postprocess_fname = f'{OUT_DIR}/data.h5'
signal_to_proc = {
    'glottal_width': glottal_width_tran,
    'time': time}
files = [f'{OUT_DIR}/{case_config(mesh_name, psub, ECOV, EBODY)}.h5' for psub in PSUBS]

# case_to_signal = postprocess_case_to_signal(postprocess_fname, files, model, signal_to_proc)

with h5py.File(postprocess_fname, mode='r') as f:
    case_to_signal = h5utils.h5_to_dict(f, {})

In [ ]:
# create resampled signals with a fixed number of samples per period
cases = list(set([key.split('/')[0] for key in case_to_signal.keys()]))

NOFFSET = 2200
NPERIOD = 5


case_to_period = {
    case:
    int(round(
        modal.estimate_fundamental_mode(
            case_to_signal[f'{case}/glottal_width'][NOFFSET:])[0]**-1
    ))
    for case in cases}

case_to_rsignal = {}
for case in cases:
    xp = np.arange(case_to_signal[f'{case}/glottal_width'].size-NOFFSET)
    fp = case_to_signal[f'{case}/glottal_width'][NOFFSET:]
    
    samples_per_period = case_to_period[f'{case}']
    num_period = int(fp.size/samples_per_period)
    
    x = np.linspace(0, num_period, num_period*N_PER_PERIOD + 1) * samples_per_period
    
    case_to_rsignal[f'{case}/glottal_width'] = np.interp(x, xp, fp)

case_to_rsignal_update = {
    f'{case}/time': 
    np.arange(case_to_rsignal[f'{case}/glottal_width'].size)/N_PER_PERIOD
    for case in cases}

case_to_rsignal.update(case_to_rsignal_update)

In [ ]:
## Post processing Hopf simulation (load the Hopf bifurcation state)
hopf_sim_fpath = f"out/hopf_state.h5"

with h5py.File(hopf_sim_fpath, mode='r') as f:
    hopf_state = bh5utils.read_block_vector_from_group(f)

## Plots

In [ ]:
## Plot transient simulations
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

for psub in PSUBS[10:]:
    # try:
    case = case_config(mesh_name, psub, ECOV, EBODY)
    t = case_to_rsignal[f'{case}/time']
    gw = case_to_rsignal[f'{case}/glottal_width']
    
    # print(gw)
    ax.plot(t, gw, label=f"$p_{{sub}}$: {psub/10:.1f} Pa", lw=1.0)
    # except:
    #     pass
    
# t0 = 0.05  
# ax.set_xlim(0.49, 0.5)
# ax.set_xlim(t0, t0+0.025)
# ax.set_xlim(0.15, 0.2)
ax.set_xlim(2, 4)

ax.set_xlabel("Time [s]")
ax.set_ylabel("Glottal width [cm]")
ax.legend(loc='lower left', bbox_to_anchor=(0, 1))

fig.tight_layout()
fig.savefig(f'fig/transient_glottal_width.png')

In [ ]:
## Plot Hopf simulations
xfp = hopf_state[:4]
xmode_real, xmode_imag = hopf_state[4:8], hopf_state[8:12]
psub = hopf_state['psub'][0]

unit_xmode_real, unit_xmode_imag = libhopf.normalize_eigenvector_amplitude(xmode_real, xmode_imag)

fig, ax = plt.subplots(1, 1)

for ampl in np.linspace(0, 10000.0, 5):
    gw = glottal_width_hopf(
        xfp.to_ndarray(),
        unit_xmode_real.to_ndarray(),
        unit_xmode_imag.to_ndarray(),
        psub, ampl, 0.0)
    ax.plot(gw, label=f"Amplitude {ampl:.2e}")
ax.set_xlabel(f"Time [period]")
ax.set_ylabel("Glottal width [cm]")
ax.legend()

fig.tight_layout()
fig.savefig("fig/onset_glottal_width_vs_amplitude.png", dpi=250)

In [ ]:
## Plot Transient vs Hopf simulations

SCALE = np.array([1e4, 1])
def sqr_err(sc_ampl_phase, gw_ref):
    ampl_phase = sc_ampl_phase*SCALE
    gw_hopf = glottal_width_hopf(
        xfp.to_ndarray(),
        unit_xmode_real.to_ndarray(),
        unit_xmode_imag.to_ndarray(),
        0.0, ampl_phase[0], ampl_phase[1])
    err = (gw_hopf-gw_ref)**2
    return jnp.sum(err)

from scipy import optimize

fig, ax = plt.subplots(1, 1)

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
psubs = np.array([400, 500, 600])*10
phase0s = np.array([0, 0, 0])*np.pi


cases = [case_config(mesh_name, psub, ECOV, EBODY) for psub in psubs]
for ii, (psub, color, case) in enumerate(zip(psubs, colors, cases)):
    t = case_to_rsignal[f'{case}/time'][:N_PER_PERIOD]
    gw_tran = case_to_rsignal[f'{case}/glottal_width'][:N_PER_PERIOD]
    
    ampl_phase0 = np.array([5e3, 0.0])
    opt_res = optimize.minimize(sqr_err, ampl_phase0/SCALE, args=(gw_tran,), jac=jax.grad(sqr_err, 0), method='BFGS')
    ampl_phase = opt_res['x'] * SCALE
    # print(res)
    
    gw_hopf0 = glottal_width_hopf(
        xfp.to_ndarray(),
        unit_xmode_real.to_ndarray(),
        unit_xmode_imag.to_ndarray(),
        psub, ampl_phase0[0], ampl_phase0[1])
    
    gw_hopf = glottal_width_hopf(
        xfp.to_ndarray(),
        unit_xmode_real.to_ndarray(),
        unit_xmode_imag.to_ndarray(),
        psub, ampl_phase[0], ampl_phase[1])
    
    ax.plot(t, gw_tran, color=color, ls='-')
    ax.plot(t, gw_hopf, color=color, ls='-.')
    ax.plot(t, gw_hopf0, color=color, ls=':')
    
ax.set_xlabel("t/T")
ax.set_ylabel("Glottal width [cm]")

from matplotlib.lines import Line2D

f0s = []
for case in cases:
    _t = case_to_signal[f'{case}/time']
    dt = _t[1]-_t[0]
    f0s.append((case_to_period[f'{case}'] * dt)**-1)

hopf_omega = hopf_state['omega'][0]
hopf_psub = hopf_state['psub'][0]
ax.legend(
    [Line2D([0], [0], color='k', ls=ls) for ls in ['-', '-.']] 
    + [Line2D([0], [0], color=color) for color in colors[:3]], 
    ['Transient', f"Onset {hopf_omega/2/np.pi:.1f} Hz {hopf_psub/10:.1f} Pa"] 
    + [f"{psub/10:.0f} Pa, {f0:.1f} Hz" for psub, f0 in zip(psubs, f0s)])

fig.savefig('fig/transient_vs_onset_gw.png', dpi=200)

In [ ]:
## Plot the onset mode shape
import dolfin as dfn
XREF = res.solid.XREF.vector()

NPHASE = 5
phases = 2*np.pi*np.linspace(0, 1, NPHASE+1)[:-1]

VERT_TO_VECDOF = dfn.vertex_to_dof_map(res.solid.forms['fspace.vector'])
CELLS = res.solid.forms['mesh.mesh'].cells()

fig, axs = plt.subplots(1, NPHASE, sharey=True, sharex=True, figsize=(10, 5))
for ax, phase in zip(axs, phases):
    ampl = 5e4*np.exp(1j*phase)
    mode_hopf_dof_order = XREF[:] + hopf_state['u'] + np.real(ampl*hopf_state['u_mode_real']) 
    
    mode_hopf_vert_order = mode_hopf_dof_order[VERT_TO_VECDOF].reshape(-1, 2)
    # print(mode_hopf_vert_order.shape)
    ax.triplot(*mode_hopf_vert_order.T, triangles=CELLS)
    ax.set_title(f"${phase/np.pi:.2f}\pi$ rad")
    
for ax in axs:
    ax.set_aspect('equal')
    ax.set_xlabel("x [cm]")
axs[0].set_ylabel("y [cm]")
    
fig.tight_layout()
fig.savefig(f'fig/hopf_mode_shape.png', dpi=200)